In [10]:
import torch
import torchvision
import scipy.io as sio
import random
import time

from collections import defaultdict

import matplotlib.pyplot as plt

import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match the input size of EfficientNet
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for pre-trained models
])

# https://pytorch.org/vision/0.12/_modules/torchvision/datasets/flowers102.html
oxford_102 = torchvision.datasets.Flowers102(root="./data", download=True, transform=transform)

oxford_102_val = torchvision.datasets.Flowers102(root="./data", download=True, split="val", transform=transform)
# print(f"Validation set size: {len(oxford_102_val)}")
oxford_102_test = torchvision.datasets.Flowers102(root="./data", download=True, split="test", transform=transform)
# print(f"Test set size: {len(oxford_102_test)}")


# setid = sio.loadmat("./data/flowers-102/setid.mat")
# print(setid.keys())

#### Part 2

from torch.utils.data import DataLoader
import torch.nn as nn
import tqdm

train_data_set = DataLoader(dataset=oxford_102, shuffle=True, batch_size=32)
test_data_set = DataLoader(dataset=oxford_102_test, batch_size=32)

efficient_net = torchvision.models.efficientnet_v2_s(progress=True)
efficient_net.train()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=efficient_net.parameters(), lr=0.001)

print('Finished Training')

Finished Training


In [13]:
#modified EFFNET

import torch
import torch.nn as nn
from torchvision.models import efficientnet_v2_s
from torchsummary import summary

# Load the original EfficientNetV2-S model
efficient_net = efficientnet_v2_s(progress=True)

# Modify the architecture
# 1. Keep the original features (convolutional layers)
# 2. Modify the classifier to have more parameters and 102 output classes
efficient_net.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=1024, bias=True),
    nn.SiLU(inplace=True),
    nn.Dropout(p=0.15, inplace=True),
    nn.Linear(in_features=1024, out_features=512, bias=True),
    nn.SiLU(inplace=True),
    nn.Dropout(p=0.1, inplace=True),
    nn.Linear(in_features=512, out_features=256, bias=True),
    nn.SiLU(inplace=True),
    nn.Linear(in_features=256, out_features=102, bias=True)
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# efficient_net.to(device)

# Print model summary
# summary(efficient_net, (3, 224, 224))

print("Fin...")

Fin...


In [11]:
# train MODIFIED for 2 epochs/ 2 GPU's

efficient_net = torch.nn.DataParallel(efficient_net)

efficient_net.train()
efficient_net.to(device)

for epoch in range(2):
    running_loss = 0.0
    time_b4_train = time.time()
    efficient_net.train()
    for images, labels in tqdm.tqdm(train_data_set):
        optimizer.zero_grad()
        outputs = efficient_net(images.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    time_after_train = time.time()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_data_set)}')
    print(f'Epoch {epoch+1}, Training Time: {time_after_train-time_b4_train}')

with torch.no_grad():
  efficient_net.eval()
  correct = 0
  total = 0
  for images, labels in tqdm.tqdm(test_data_set):
    predictions = efficient_net(images.to(device))
    _, predicted = torch.max(predictions, 1)
    total += labels.size(0)  # Total number of labels
    correct += (predicted == labels.to(device)).sum().item()  # Count correct predictions

  print(f'Epoch {epoch+1}, Test Accuracy: {(correct/total) *100}')

print('Finished Training')

100%|██████████| 32/32 [00:27<00:00,  1.17it/s]


Epoch 1, Loss: 5.530396819114685
Epoch 1, Training Time: 27.43110966682434


100%|██████████| 32/32 [00:12<00:00,  2.59it/s]


Epoch 2, Loss: 4.398523792624474
Epoch 2, Training Time: 12.348190069198608


100%|██████████| 193/193 [01:57<00:00,  1.64it/s]

Epoch 2, Test Accuracy: 2.228004553585949
Finished Training


In [14]:
# MODIFIED-NET Single GPU

print("Let's use", torch.cuda.device_count(), "GPUs!")

# train MODIFIED for 2 epochs/ 2 GPU's

# efficient_net = torch.nn.DataParallel(efficient_net)

efficient_net.train()
efficient_net.to(device)

for epoch in range(2):
    running_loss = 0.0
    time_b4_train = time.time()
    efficient_net.train()
    for images, labels in tqdm.tqdm(train_data_set):
        optimizer.zero_grad()
        outputs = efficient_net(images.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    time_after_train = time.time()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_data_set)}')
    print(f'Epoch {epoch+1}, Training Time: {time_after_train-time_b4_train}')

with torch.no_grad():
  efficient_net.eval()
  correct = 0
  total = 0
  for images, labels in tqdm.tqdm(test_data_set):
    predictions = efficient_net(images.to(device))
    _, predicted = torch.max(predictions, 1)
    total += labels.size(0)  # Total number of labels
    correct += (predicted == labels.to(device)).sum().item()  # Count correct predictions

  print(f'Epoch {epoch+1}, Test Accuracy: {(correct/total) *100}')

print('Finished Training')


Let's use 2 GPUs!


100%|██████████| 32/32 [00:14<00:00,  2.28it/s]


Epoch 1, Loss: 4.626077190041542
Epoch 1, Training Time: 14.053879261016846


100%|██████████| 32/32 [00:13<00:00,  2.39it/s]


Epoch 2, Loss: 4.626089662313461
Epoch 2, Training Time: 13.41404104232788


100%|██████████| 193/193 [01:03<00:00,  3.06it/s]

Epoch 2, Test Accuracy: 1.4311270125223614
Finished Training


In [4]:
efficient_net = torchvision.models.efficientnet_v2_s(progress=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=efficient_net.parameters(), lr=0.001)

print("Let's use", torch.cuda.device_count(), "GPUs!")
# efficient_net = torch.nn.DataParallel(efficient_net)

efficient_net.train()
efficient_net.to(device)



for epoch in range(2):
    running_loss = 0.0
    time_b4_train = time.time()
    efficient_net.train()
    for images, labels in tqdm.tqdm(train_data_set):
        optimizer.zero_grad()
        outputs = efficient_net(images.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    time_after_train = time.time()
    with torch.no_grad():
      efficient_net.eval()
      correct = 0
      total = 0
      for images, labels in tqdm.tqdm(test_data_set):
        predictions = efficient_net(images.to(device))
        _, predicted = torch.max(predictions, 1)
        total += labels.size(0)  # Total number of labels
        correct += (predicted == labels.to(device)).sum().item()  # Count correct predictions

      print(f'Epoch {epoch+1}, Test Accuracy: {(correct/total) *100}')

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_data_set)}')
    print(f'Epoch {epoch+1}, Training Time: {time_after_train-time_b4_train}')

print('Finished Training')

Let's use 1 GPUs!


100%|██████████| 193/193 [00:45<00:00,  4.21it/s]


Epoch 1, Test Accuracy: 0.35778175313059035
Epoch 1, Loss: 5.722531795501709
Epoch 1, Training Time: 11.513773441314697


100%|██████████| 193/193 [00:49<00:00,  3.92it/s]

Epoch 2, Test Accuracy: 3.171247357293869
Epoch 2, Loss: 4.5821602791547775
Epoch 2, Training Time: 9.57150650024414
Finished Training


In [18]:
# DATA AUGMENTATION

#### 5. Transform part

from torch.utils.data import DataLoader
import torch.nn as nn
import tqdm

import torch
import torchvision
import scipy.io as sio
import random
import time

import numpy as np
from PIL import Image

from collections import defaultdict

import matplotlib.pyplot as plt

import torchvision.transforms as transforms


# Custom Channel Inversion transformation
class ChannelColorInversion(object):
    def __init__(self, randomPixelChange, channels=(0, 1, 2)):  # By default, inverts all channels (R, G, B)
        self.channels = channels
        self.randomPixelChange = randomPixelChange

    def __call__(self, img):
        if isinstance(img, torch.Tensor):
            img = transforms.ToPILImage()(img)

        np_img = np.array(img)

        # Invert specified channels
        for channel in self.channels:
            # np_img[..., channel] = 255 - np_img[..., channel]
            np_img[..., channel] = (self.randomPixelChange + np_img[..., channel])%255


        return Image.fromarray(np_img.astype('uint8'))

randomPixel = random.randint(-2, 2)

# Apply the custom transformation and load the dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize the image
    ChannelColorInversion(randomPixel, channels=(0, 2)),  # Invert only the Red and Blue channels
    transforms.ToTensor(),  # Convert back to Tensor
])

# https://pytorch.org/vision/0.12/_modules/torchvision/datasets/flowers102.html
oxford_102 = torchvision.datasets.Flowers102(root="./data", download=True, transform=transform)

oxford_102_val = torchvision.datasets.Flowers102(root="./data", download=True, split="val", transform=transform)
# print(f"Validation set size: {len(oxford_102_val)}")
oxford_102_test = torchvision.datasets.Flowers102(root="./data", download=True, split="test", transform=transform)
# print(f"Test set size: {len(oxford_102_test)}")

from torch.utils.data import DataLoader
import torch.nn as nn
import tqdm

train_data_set = DataLoader(dataset=oxford_102, shuffle=True, batch_size=32)
test_data_set = DataLoader(dataset=oxford_102_test, batch_size=32)

efficient_net = torchvision.models.efficientnet_v2_s(progress=True)
efficient_net.train()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=efficient_net.parameters(), lr=0.001)

print("Let's use", torch.cuda.device_count(), "GPUs!")
efficient_net = torch.nn.DataParallel(efficient_net)

efficient_net.train()
efficient_net.to(device)

# correctly classified image and label
correct_classified_image = []
correct_classified_label = []

incorrect_classified_image = []
incorrect_classified_label = []

max_correct = 2
max_incorrect = 2

for epoch in range(2):
    running_loss = 0.0
    time_b4_train = time.time()
    efficient_net.train()
    for images, labels in tqdm.tqdm(train_data_set):
        optimizer.zero_grad()
        outputs = efficient_net(images.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    time_after_train = time.time()
    with torch.no_grad():
      efficient_net.eval()
      correct = 0
      total = 0
      for images, labels in tqdm.tqdm(test_data_set):
        predictions = efficient_net(images.to(device))
        _, predicted = torch.max(predictions, 1)
        total += labels.size(0)  # Total number of labels
        correct += (predicted == labels.to(device)).sum().item()  # Count correct predictions

        if max_correct > 0 and (predicted == labels.to(device)).sum().item() > 0:
          correct_classified_image.append(images)
          correct_classified_label.append(labels)
          max_correct -= 1
        if max_incorrect > 0 and (predicted == labels.to(device)).sum().item() > 0:
          max_incorrect -= 1
          incorrect_classified_image.append(images)
          incorrect_classified_label.append(labels)

      print(f'Epoch {epoch+1}, Test Accuracy: {(correct/total) *100}')

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_data_set)}')
    print(f'Epoch {epoch+1}, Training Time: {time_after_train-time_b4_train}')

print('Finished Training')


Let's use 2 GPUs!


100%|██████████| 193/193 [00:53<00:00,  3.62it/s]


Epoch 1, Test Accuracy: 0.3252561392096276
Epoch 1, Loss: 5.718189403414726
Epoch 1, Training Time: 11.17623257637024


100%|██████████| 193/193 [00:53<00:00,  3.64it/s]

Epoch 2, Test Accuracy: 0.9107171897869573
Epoch 2, Loss: 4.536223724484444
Epoch 2, Training Time: 10.823798894882202
Finished Training
